<a href="https://colab.research.google.com/github/AnDDoanf/Biblical_NLP_task/blob/master/quesans/QABible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import modules

In [ ]:
!pip install bs4
!pip install html2text
!pip install mechanize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Crawl data from GOT

In [ ]:
import mechanize
import nltk
from bs4 import BeautifulSoup
import re

def getdata(url):
  br = mechanize.Browser()
  br.set_handle_robots(False)
  br.addheaders = [('User-agent', 'Firefox')]
  html = br.open(url).read().decode('utf-8')
  soup = BeautifulSoup(html)
  text2 = soup.get_text()
  question = text2[text2.index("Question"):text2.index("Answer")].replace("\n", "").replace("Question: ", "").replace("\"","").strip()
  answer = text2[text2.index("Answer"):text2.index("www.GotQuestions.org")].replace("\n", "").replace("Answer: ", "").strip()
  return [question, answer]

In [ ]:
import pandas as pd
lst = []
for i in range (5, 3551):
  temp = getdata("https://printer.gotquestions.net/GeneratePF?articleId="+str(i))
  lst.append(temp)


In [ ]:
data = pd.DataFrame(lst, columns= ["Question", "Answer"])

In [ ]:
data[:10]

,Question,Answer
0,How can I know if something is a sin?,There are two issues involved in this question...
1,How can I know for sure that I will go to heav...,Do you know for certain that you have eternal ...
2,What is the key to truly knowing God?,Within all of us there exists a strong desire ...
3,What is koinonia?,Koinonia is a Greek word that occurs 20 times ...
4,"Are apparitions of Mary, such as Lady Fatima, ...","In Catholic tradition, there are many reported..."
5,"What are infralapsarianism, sublapsarianism, a...","These three theological terms, discussed among..."
6,What are the Christian themes in The Last Battle?,"Titled accordingly, The Last Battle is the fin..."
7,Did the people in the Bible have last names?,People in Bible times did not really have last...
8,What is the Latin Vulgate Bible?,“The Vulgate” is the popular name given to the...
9,What is the Latter Rain Movement?,The Latter Rain Movement is an influence withi...


In [ ]:
data[:10].to_csv("QA_Bible_Dataset.csv", encoding="utf-8")

### Crawl data from Enduring World

In [ ]:
def getdata2(url):
  br = mechanize.Browser()
  br.set_handle_robots(False)
  br.addheaders = [('User-agent', 'Firefox')]
  html = br.open(url).read().decode('utf-8')
  soup = BeautifulSoup(html)
  start = str(soup).find("<h3>")
  end = str(soup).find('Bible Commentary Quick Navigation')
  text2 = BeautifulSoup(str(soup)[start:end]).get_text()
  return text2

In [ ]:
import os
import shutil

datadir = "bible_commentary_dataset"
if os.path.exists(datadir):
  shutil.rmtree(datadir)
os.mkdir(datadir)

# Crawl mannually with joshua and 1-chronicles
OTbooks = [["genesis", 50], ["exodus", 40], ["leviticus", 27], ["numbers", 36], ["deuteronomy", 34], ["joshua", 14], 
         ["judges", 21], ["ruth", 4], ["1-samuel", 31], ["2-samuel", 24], ["1-kings", 22], ["2-kings", 25], 
         ["1-chronicles", 3], ["2-chronicles", 36], ["ezra", 10], ["nehemiah", 13], ["esther", 10], ["job", 42], 
         ["psalm", 150], ["proverbs", 31], ["ecclesiastes", 12], ["song-of-solomon", 8], ["isaiah", 66], ["jeremiah", 52],
         ["lamentations", 5], ["ezekiel", 48], ["daniel", 12], ["hosea", 14], ["joel", 3], ["amos", 9],
         ["obadiah", 1], ["jonah", 4], ["micah", 1], ["nahum", 3], ["habakkuk", 3], ["zephaniah", 3],
         ["haggai", 2], ["zechariah", 14], ["malachi", 4]]


for book_index, book in enumerate(OTbooks):
  if book_index < 9:
    bookdir = os.path.join(datadir,"0"+str(book_index+1)+"_"+book[0])
  else:
    bookdir = os.path.join(datadir,str(book_index+1)+"_"+book[0])
  if os.path.exists(bookdir):
    shutil.rmtree(bookdir)
  os.mkdir(bookdir)
  for chapter in range (1, book[1]+1):
    temp = getdata2("https://enduringword.com/bible-commentary/"+book[0]+"-"+str(chapter)+"/")
    if chapter < 10:
      with open(os.path.join(bookdir,book[0]+"00"+str(chapter)+".txt"), "w") as f:
          f.write(temp)
    elif chapter < 100:
      with open(os.path.join(bookdir,book[0]+"0"+str(chapter)+".txt"), "w") as f:
          f.write(temp)
    else: 
      with open(os.path.join(bookdir,book[0]+str(chapter)+".txt"), "w") as f:
          f.write(temp)

In [ ]:
!zip -r /content/bible_commentary_dataset.zip /content/bible_commentary_dataset

### Load data

In [ ]:
def txt2paragraph(file):
  text = open(file).read()
  replace_list = ['A. ', 'B.', 'C.', 'D.', 'E.', 'F.', 'G.', 'H.', 'I.', 'J.', 'K.', 'L.', 'M.', 'N.', 'O.', 'P.', 
                  '\n1.','\n2.','\n3.','\n4.', '\n5.','\n6.', '\n7.','\n8.' '\n9.','\n10.',
                  '\n·', '\na.', '\nb.', '\nc.', '\nd.', '\ne.', '\nf.', '\ng.', '\nh.', '\ni.', '\nj.', '\nk.', '\nl.', '\nm.', '\nn.', '\no.', '\np.',
                  '\ni.', '\nii.', '\niii.', '\niv.', '\nv.', '\nvi.', '\nvii.', '\nviii.', '\nix.', '\nx.', '\nxi.']
  
  import re
  clean = re.sub(r'|'.join(map(re.escape, replace_list)), '', text)

  clean = re.sub('.\n', '. ', clean)
  clean = re.sub('\n', '', clean)
  return clean.split(".")[:-3]

In [ ]:
lst = []
datasetdir = "/content/bible_commentary_dataset"
for bookdir in sorted(os.listdir(datasetdir)):
  temp = []
  for chapterdir in sorted(os.listdir(os.path.join(datasetdir, bookdir))):
    temp.append(txt2paragraph(os.path.join(datasetdir, bookdir, chapterdir)))
  lst.append(temp)